In [1]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer

In [2]:
webtoon=pd.read_csv('webtoon.csv')
toon = pd.DataFrame(webtoon)
toon.describe

<bound method NDFrame.describe of           id              title      author      genre  \
0     756056       가난을 등에 업은 소녀  B급달궁 / 오은지   스토리, 로맨스   
1     670144               가담항설          랑또   스토리, 판타지   
2     732071       가령의 정체불명 이야기          가령  옴니버스, 드라마   
3     703844              가비지타임         2사장   스토리, 스포츠   
4     785701             가상&RPG         주다현   스토리, 판타지   
...      ...                ...         ...        ...   
2095  675393    [드라마원작] 한번 더 해요     미티 / 구구   스토리, 드라마   
2096  687921    [드라마원작] 아일랜드 2부   윤인완 / 양경일   스토리, 스릴러   
2097  677536  [드라마원작] 내 ID는 ...         기맹기   스토리, 로맨스   
2098  678500    [드라마원작] 아일랜드 1부   윤인완 / 양경일   스토리, 스릴러   
2099   67235  [드라마원작] 지금 우리 ...         주동근   스토리, 스릴러   

                                            description  rating  \
0     흔하디 흔한 재벌후계자와 캔디도 울고 갈 박복한 가난소녀의 파란만장 동거기!인기작 ...    9.13   
1     이번 주인공은 돌이다!돌이지만 동료도 모으고 악당도 물리친다!랑또 작가표 동양 판타...    9.98   
2     어느 날... 인어가 내게 말을 걸어왔다. 눈을 떠보니 총구가 있었다. 그리고 몸이...    9.95   
3

In [3]:
desc = toon['description']
print(desc[:4])

0    흔하디 흔한 재벌후계자와 캔디도 울고 갈 박복한 가난소녀의 파란만장 동거기!인기작 ...
1    이번 주인공은 돌이다!돌이지만 동료도 모으고 악당도 물리친다!랑또 작가표 동양 판타...
2    어느 날... 인어가 내게 말을 걸어왔다. 눈을 떠보니 총구가 있었다. 그리고 몸이...
3      한국 최초 한국형 고교스포츠 웹툰!열혈따윈 개나 줘, 낙오자들 뿐인 농구부의 운명은?
Name: description, dtype: object


In [4]:
text=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]","",desc[0])
print(text)

흔하디 흔한 재벌후계자와 캔디도 울고 갈 박복한 가난소녀의 파란만장 동거기인기작 다세포소녀의 웹툰판


In [5]:
okt=Okt()
review_text=okt.nouns(text)
print(review_text)

['하디', '재벌', '후계', '캔디', '울', '가난', '소녀', '파란만장', '동거', '인기', '작', '다세포', '소녀', '웹툰', '판']


In [6]:
def preprocessing(text, okt, remove_stopwords = False, stop_words = []):
    # text : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    desc_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", text)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    word_desc = okt.nouns(desc_text)

    return word_desc

In [7]:
clean_desc=[]
stop_words=set(['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한','그'])
for desc in toon['description']:
    # 빈 데이터에서 멈추지 않도록 문자열인 경우에만 진행
    if type(desc)==str:
        clean_desc.append(preprocessing(desc, okt, remove_stopwords=True, stop_words=stop_words))
    else:
        # stirng이 아니면 비어있는 값 추가
        clean_desc.append([])

In [8]:
print(clean_desc[:5])

[['하디', '재벌', '후계', '캔디', '울', '가난', '소녀', '파란만장', '동거', '인기', '작', '다세포', '소녀', '웹툰', '판'], ['이번', '주인공', '돌이', '돌이', '동료', '악당', '물리', '또', '가표', '동양', '판타지', '소년만화'], ['날', '인어', '말', '눈', '총구', '몸', '연기', '시작', '판타지', '비현실적', '우리', '회색', '빛', '이야기'], ['한국', '최초', '한국', '고교', '스포츠', '웹툰', '혈', '윈', '개', '낙', '오자', '농구부', '운명'], ['사람', '좀', '게임', '집', '바깥', '제리', '전투', '번', '에픽', '무기', '귀속', '가상현실', '액션', '모브사', '온라인', '그', '인물', '군상']]


In [27]:
num_lists = len(clean_desc)
num_elements = sum(len(inner_list) for inner_list in clean_desc)
# 토큰의 평균 갯수
average = num_elements / num_lists
print(average)

19.339615384615385


In [10]:
# 챌린지 웹툰을 테스트 데이터로 사용
chal_webtoon=pd.read_csv('webtoon_challenge.csv')
chal_toon = pd.DataFrame(chal_webtoon)

clean_chal_desc=[]
stop_words=set(['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한','그'])
for desc in chal_toon['description'][:500]:
    # 빈 데이터에서 멈추지 않도록 문자열인 경우에만 진행
    if type(desc)==str:
        clean_chal_desc.append(preprocessing(desc, okt, remove_stopwords=True, stop_words=stop_words))
    else:
        # stirng이 아니면 비어있는 값 추가
        clean_chal_desc.append([])

In [31]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(clean_desc)
train_sequences=tokenizer.texts_to_sequences(clean_desc)
test_sequences=tokenizer.texts_to_sequences(clean_chal_desc)

# 단어 사전 형태
word_vocab=tokenizer.word_index

# 문장 최대 길이
MAX_SEQUENCE_LENGTH=19

# 학습 데이터를 벡터화
train_inputs=pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
# 학습 데이터의 라벨
train_labels=np.array([toon['rating']])

# 평가 데이터를 벡터화
test_inputs=pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
# 평가 데이터의 라벨
test_labels=np.array(chal_toon['rating'][:500])

In [32]:
train_labels = train_labels.reshape(-1,)
test_labels = test_labels.reshape(-1,)

In [41]:
test_labels = test_labels[:500]

In [33]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_labels = scaler.fit_transform(toon['rating'].values.reshape(-1,1))
test_labels = scaler.transform(chal_toon['rating'].values.reshape(-1,1))

In [34]:
print(len(word_vocab))

8679


In [35]:
print(train_inputs[:9])

[[4457  277  682 4458  556 1740   35 1235  243  114  258 4459   35   56
   824    0    0    0    0]
 [ 244   41 1072 1072  428  598  933  111 4460 2951   52 4461    0    0
     0    0    0    0    0]
 [   9 2188   66   40 4462   71  391    3   52 4463   49 2189  392    6
     0    0    0    0    0]
 [ 174  683  174  825  599   56 1073 4464  278 4465 2952 2953   61    0
     0    0    0    0    0]
 [   5  517   31   50 2954 4466  934  124 4467  935 2955 1741   94 4468
  1074    1  318 2956    0]
 [  88 1742   28  207  936 4469   29 4470 1743  485 1743 4471 1743  334
    38    0    0    0    0]
 [2957 2190  486  752 4472 4473  346    6 4474   11  203  450   38    0
     0    0    0    0    0]
 [4475   21   99 4476   50  243    3 1744   85  294   12    3   82   13
    82   13  367  107   82]
 [ 231   77  826   82  827   86   82    6    0    0    0    0    0    0
     0    0    0    0    0]]


In [36]:
print(train_inputs.shape)
print(train_labels.shape)

(2100, 19)
(2100, 1)


In [37]:
from sklearn.model_selection import train_test_split
# 하이퍼 파라미터
EMB_SIZE = 128
BATCH_SIZE = 16
NUM_EPOCHS = 1

input_train, input_eval, label_train, label_eval=train_test_split(train_inputs, train_labels, test_size=0.1)

In [39]:
print(test_inputs.shape, test_labels.shape)

(500, 19) (3192, 1)


In [42]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

# Define model
model = Sequential()
model.add(Embedding(len(word_vocab) + 1, EMB_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(EMB_SIZE))
model.add(Dense(1))  # No activation function for regression

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

# Train model
model.fit(train_inputs, train_labels, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_split=0.1)
loss, mae = model.evaluate(test_inputs, test_labels)
print(f'Test Mean Absolute Error: {mae}')


16/16 [==============================] - 0s 3ms/step - loss: 0.6321 - mean_absolute_error: 0.4042
Test Mean Absolute Error: 0.40415874123573303


In [43]:
# Make predictions
predictions = model.predict(test_inputs)

# Compute custom accuracy
accuracy = np.mean(np.abs(predictions - test_labels) < 0.5)
print(f'Accuracy: {accuracy}')

16/16 [==============================] - 0s 3ms/step
Accuracy: 0.826
